# Modeling

Nous allons dans ce notebook étabilir un modèle de recommandation, nous allons utiliser sklearn pour mener à bien ce projet.

### Import des librairies

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import implicit
import scipy.sparse as sparse
import pickle
from random import randrange

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

C:\tools\Anaconda3\envs\P9\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import des données

In [2]:
articles_categories_all = pd.read_csv('data/articles_categories.csv')
articles_metadata = pd.read_csv("data/articles_metadata.csv")
clicks_sample = pd.read_csv("data/clicks_sample.csv")
df_embeddings = pd.read_csv("data/df_embeddings.csv")

## Calcul de similarité 

### Content-Based Filtering

Le **Content-Based Filtering** est une méthode de filtrage basée sur le contenu qui permet aux systèmes de recommandation d'analyser des caractéristique (par exemple, le genre, l'acteur, le réalisateur d'un film) et de déterminer les préférences des utilisateurs à partir de ces caractéristiques. Le système peut ensuite utiliser ces préférences pour générer des recommandations personnalisées pour chaque utilisateur.

Nous allons ici calculer la similarité entre les différents articles, nous allons utiliser seulement les variables pertinentes.

In [3]:
# Supprimer les doublons
articles_categories = articles_categories_all.drop_duplicates(
    subset='click_article_id')

In [4]:
%%time
# Extraire les features
features = [
    'click_country', 'click_article_id', 'words_count', 'click_region',
    'category_id', 'publisher_id'
]

# Calculer la similarité entre l'article lu et les articles recommandés
similarities = cosine_similarity(articles_categories[features])

CPU times: total: 1min 7s
Wall time: 1min 34s


In [5]:
# Récupérer l'index de l'article lu
article_id = articles_categories["click_article_id"].iloc[0]
clicked_article_index = articles_categories[
    articles_categories['click_article_id'] == article_id].index[0]

In [6]:
# Trouver les 5 articles les plus similaires
similar_article_indices = similarities[clicked_article_index].argsort(
)[-6:-1][::-1]

In [7]:
# Récupérer les articles à partir de leur ID
similar_articles_ids = [
    articles_categories.index[i] for i in similar_article_indices
]
similar_articles = [articles_categories.loc[id] for id in similar_articles_ids]

In [8]:
# Récupérer les scores de similarité
similarity_scores = similarities[clicked_article_index][
    similar_article_indices]

In [9]:
# Afficher les 5 articles les plus similaires et leurs scores de similarité
for article, score in zip(similar_articles, similarity_scores):
    print('Article :', article['click_article_id'])
    print('Score de similarité :', score)
    print()

Article : 157560
Score de similarité : 0.9999999999611299

Article : 158513
Score de similarité : 0.9999999998926756

Article : 158132
Score de similarité : 0.9999999998766111

Article : 156147
Score de similarité : 0.999999999825046

Article : 168887
Score de similarité : 0.9999999997331097



Et les 5 articles les moins similaires.

In [10]:
similar_article_indices = similarities[clicked_article_index].argsort(
)[0:5][::-1]
# Récupérer les articles à partir de leur ID
similar_articles_ids = [
    articles_categories.index[i] for i in similar_article_indices
]
similar_articles = [articles_categories.loc[id] for id in similar_articles_ids]

# Récupérer les scores de similarité
similarity_scores = similarities[clicked_article_index][
    similar_article_indices]

# Afficher les 5 articles les plus similaires et leurs scores de similarité
for article, score in zip(similar_articles, similarity_scores):
    print('Article :', article['click_article_id'])
    print('Score de similarité :', score)
    print()

Article : 94
Score de similarité : 0.41846173841006656

Article : 81
Score de similarité : 0.37381691561598884

Article : 69
Score de similarité : 0.23239586066989854

Article : 27
Score de similarité : 0.17547506853040948

Article : 3
Score de similarité : 0.014822440893156607



### Collaborative Filtering

Le **Collaborative Filtering** est une méthode de filtrage fondée sur les opinions d'autres utilisateurs. 

Les systèmes de recommandation qui utilisent ce type de filtrage recueillent les préférences et les opinions des utilisateurs sur différents objets de données et utilisent ces informations pour créer des groupes d'utilisateurs avec des goûts similaires. Ils peuvent ensuite utiliser ces groupes pour générer des recommandations personnalisées pour chaque utilisateur en fonction des préférences des autres membres du groupe.

Le soucis ici, c'est que nous n'avons pas de rating (note d'articles par exemple ou de 'pouce en l'air'), nous devons donc d'abord établir un score pour chaque article en fonction de certaines variable comme le nombre de clique par article, l'ancienneté de l'article.

In [45]:
def df_rating(articles_metadata, dataframe_all):
    # Fusionne les données des articles et des clics
    user_interactions_df = pd.merge(articles_metadata,
                                    dataframe_all,
                                    left_on=['article_id'],
                                    right_on=['click_article_id'])
    
    # Trie les données par user_id et date de session
    user_interactions_df.sort_values(by=['user_id', 'session_start'],
                                     ascending=True,
                                     inplace=True)
    
    # Supprime les colonnes qui ne sont pas nécessaires
    user_interactions_df.drop([
        'click_article_id', 'session_start', 'session_size', 'click_timestamp'
    ],
                              axis=1,
                              inplace=True)
    
    # Sélectionne les colonnes qui nous intéressent
    user_interactions_df = user_interactions_df[[
        'user_id', 'article_id', 'session_id'
    ]]
    
    # Compte le nombre d'interactions entre l'utilisateur et l'article
    user_interactions_df = user_interactions_df.groupby(
        by=['user_id', 'article_id'], as_index=False).agg('count')
    
    # Renomme la colonne
    user_interactions_df.rename(columns={"session_id": "interactionStrength"},
                                inplace=True)
    
    # Trie par interactionStrength
    user_interactions_df.sort_values(by=['interactionStrength'],
                                     ascending=False)
    
    return user_interactions_df

In [46]:
collab_data_df = articles_categories_all
collab_data_df = df_rating(articles_metadata, collab_data_df)
collab_data_df.head()

,user_id,article_id,interactionStrength
0,0,68866,1
1,0,87205,1
2,0,87224,1
3,0,96755,1
4,0,157541,1


In [13]:
collab_data_df['article_id'].nunique()

46033

Nous allons encoder nos identifiant d'article, cela va faciliter la suite puisque nous aurons besoin d'avoir une valeur inférieur à la longueur de notre csr_matrix.

In [14]:
# créons un LabelEncoder
le = LabelEncoder()

# appliquons le LabelEncoder à la colonne article_id
collab_data_df['article_id_encode'] = le.fit_transform(
    collab_data_df['article_id'])

collab_data_df.to_csv('data/collab_data_df.csv', index=False)

collab_data_df

,user_id,article_id,interactionStrength,article_id_encode
0,0,68866,1,9349
1,0,87205,1,12368
2,0,87224,1,12382
3,0,96755,1,13647
4,0,157541,1,21330
...,...,...,...,...
2950705,322894,168401,1,23140
2950706,322895,63746,1,8411
2950707,322895,289197,1,38069
2950708,322896,30760,1,2999


In [15]:
%%time
sparse_user_item = sparse.csr_matrix(
    (collab_data_df['interactionStrength'].astype(float),
     (collab_data_df['user_id'], collab_data_df['article_id_encode'])))

CPU times: total: 31.2 ms
Wall time: 67 ms


In [16]:
# %%time
# colab_model = implicit.als.AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
# colab_model.fit(sparse_user_item)
colab_model = pickle.load(open("colab_model.sav", 'rb'))

In [17]:
USER_ID = collab_data_df['user_id'].iloc[0]

item_ids, scores = colab_model.recommend(USER_ID,
                                         sparse_user_item[USER_ID],
                                         N=5,
                                         filter_already_liked_items=True)

In [18]:
# Export le model
# filename = 'colab_model.sav'
# pickle.dump(colab_model, open(filename, 'wb'))

In [19]:
(item_ids, scores)

(array([38579, 37233, 21945, 30903, 21621]),
 array([0.5715908 , 0.41498896, 0.3296845 , 0.31086028, 0.3007176 ],
       dtype=float32))

In [20]:
# Vérification si les recommandations sont bien des recommandations d'articles et non d'user
np.unique(collab_data_df[collab_data_df['article_id_encode'].isin(item_ids)]['article_id_encode'])

array([21621, 21945, 30903, 37233, 38579], dtype=int64)

#### Score

Nous allons ici utiliser les catégories pour évaluer notre modèle, le but ici est de comparer la catégorie des articles recommandés, aux catégories lu par l'utilisateur.

In [21]:
# Etant donné que les valeurs utilisées pour la recommandation sont les id_d'articles encodés,
# nous devons en récuperer la véritable id des articles
item_ids_list = item_ids.tolist()
item_ids_article_id = []
for i in item_ids_list:
    try:
        item_ids_article_id.append(collab_data_df.loc[collab_data_df['article_id_encode'] == i,
                           'article_id'].values[0])
    except:
        print(i)

In [22]:
item_ids_article_id

[293114, 284985, 160940, 233658, 158906]

In [23]:
# Place en cle les id d'article et en valeur leurs catégories
category_id_article_recommande = {}

for i in item_ids_article_id:
    try:
        category_id_article_recommande[i] = articles_categories_all.loc[
            articles_categories_all['click_article_id'] == i,
            'category_id'].values[0]
    except:
        print(i)

In [30]:
# Liste des catégories des articles lu par l'utilisateur
user_consulted_categories = articles_categories_all.query(
    'user_id == ' + str(USER_ID))['category_id'].tolist()
user_consulted_categories = set(user_consulted_categories)
user_consulted_categories

{136, 186, 209, 281, 375, 431}

In [31]:
# Créer un dictionnaire pour stocker le nombre d'articles correspondant aux catégories consultées par l'utilisateur
articles_correspondants = {}

# Compter le nombre d'articles correspondant aux catégories consultées par l'utilisateur
for article_id, category_id in category_id_article_recommande.items():
    if category_id in user_consulted_categories:
        if category_id in articles_correspondants:
            articles_correspondants[category_id] += 1
        else:
            articles_correspondants[category_id] = 1

# Calculer le score en pourcentage
total_articles = len(category_id_article_recommande)
correspondant_articles = sum(articles_correspondants.values())
score_pourcentage = (correspondant_articles / total_articles) * 100

print(score_pourcentage)

60.0


Nous devons maintenant établir un score globale sur tous les users.

In [32]:
def score_total(user_id):
    USER_ID = user_id

    item_ids, scores = colab_model.recommend(USER_ID,
                                             sparse_user_item[USER_ID],
                                             N=5,
                                             filter_already_liked_items=True)

    item_ids_list = item_ids.tolist()
    item_ids_article_id = []
    for i in item_ids_list:
        try:
            item_ids_article_id.append(
                collab_data_df.loc[collab_data_df['article_id_encode'] == i,
                                   'article_id'].values[0])
        except:
            print(i)
    # Place en cle les id d'article et en valeur leurs catégories
    category_id_article_recommande = {}

    for i in item_ids_article_id:
        try:
            category_id_article_recommande[i] = articles_categories_all.loc[
                articles_categories_all['click_article_id'] == i,
                'category_id'].values[0]
        except:
            print(i)

    # Créer un dictionnaire pour stocker le nombre d'articles correspondant aux catégories consultées par l'utilisateur
    articles_correspondants = {}

    # Compter le nombre d'articles correspondant aux catégories consultées par l'utilisateur
    for article_id, category_id in category_id_article_recommande.items():
        if category_id in user_consulted_categories:
            if category_id in articles_correspondants:
                articles_correspondants[category_id] += 1
            else:
                articles_correspondants[category_id] = 1

    # Calculer le score en pourcentage
    total_articles = len(category_id_article_recommande)
    correspondant_articles = sum(articles_correspondants.values())
    score_pourcentage = (correspondant_articles / total_articles) * 100

    return score_pourcentage

In [38]:
%%time
# Initialiser une liste pour stocker les scores de chaque user_id
scores = []

# Parcourir tous les user_id et calculer le score de chaque user_id
for user_id in collab_data_df['user_id'].head(1000000).unique():
    score = score_total(user_id)
    scores.append(score)

# Calculer le score moyen
score_moyen = sum(scores) / len(scores)

print('Le score moyen est:', score_moyen)

Le score moyen est: 33.53863033738685
CPU times: total: 45min 20s
Wall time: 57min


# Fonction de recommandation (collaborative filtering)

In [47]:
def recommandation_5(USER_ID):
    # import du modele
    colab_model = pickle.load(open("colab_model.sav", 'rb'))
    
    # import de la dataframe
    collab_data_df = pd.read_csv("data/collab_data_df.csv")
    
    sparse_user_item = sparse.csr_matrix(
    (collab_data_df['interactionStrength'].astype(float),
     (collab_data_df['user_id'], collab_data_df['article_id_encode'])))
    item_ids, scores = colab_model.recommend(USER_ID,
                                             sparse_user_item[USER_ID],
                                             N=5,
                                             filter_already_liked_items=True)
    
    # Récupère la vrai id des articles et non les encodés
    item_ids_list = item_ids.tolist()
    item_ids_article_id = []
    for i in item_ids_list:
        try:
            item_ids_article_id.append(collab_data_df.loc[collab_data_df['article_id_encode'] == i,
                               'article_id'].values[0])
        except:
            print(i)

    article_recommandes = {}
    for i in zip(item_ids_article_id, scores):
        article_recommandes[str(i[0])] = str(i[1])
    
    
    return article_recommandes

In [48]:
recommandation_5(randrange(500))

{'225019': '0.58908427',
 '202557': '0.40266645',
 '225055': '0.39903554',
 '271261': '0.3558324',
 '31520': '0.35030746'}